In [1]:
def get_input(day):
    filename = 'input/day{}.txt'.format(day)
    with open(filename) as f:
        return [line.strip() for line in f]

# [Day 1: Inverse Captcha](http://adventofcode.com/2017/day/1)

**Task:** Given a circular sequence of digits, find the sum of all digits that match the next digit in the list.

**Examples**:
* `1122` -> `3`
* `1111` -> `4`
* `91212129` -> `9`

In [2]:
def solve_captcha(sequence):
    total = 0
    previous_number = sequence[-1]
    for number in sequence:
        if previous_number == number:
            total += int(previous_number)
        previous_number = number
    return total

solve_captcha(get_input(1)[0])

1341